In [46]:
import pandas as pd
import numpy as np
import re
import os
import timeit
os.chdir("C:/Users/linhd/OneDrive/8th Semester/MIS 4596/Code Samples")

In [47]:
df = pd.read_csv('BL.csv')
df.head(2)

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.


In [48]:
df.columns #Check the column names

Index(['Identifier', 'Edition Statement', 'Place of Publication',
       'Date of Publication', 'Publisher', 'Title', 'Author', 'Contributors',
       'Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Flickr URL', 'Shelfmarks'],
      dtype='object')

In [49]:
to_drop = ['Edition Statement', 'Corporate Author', 'Corporate Contributors','Former owner','Engraver','Contributors', 'Issuance type','Shelfmarks']
df = df.drop(to_drop, axis=1)

# Alternatively
# df_lean = df_df[df_df.columns[df_df.columns.isin(to_drop)]]
df.head(2) #Check again to see those columns are dropped.

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...


In [50]:
#Set Identifier as the Index
df['Identifier'].is_unique #Finding out if the Identifier is actually is_unique
df.set_index('Identifier', inplace=True) #so that it is directly assigned to df itself, without have to say df = df.set_index(..)

#Count how many unique types of data exist within this dataset. As the dataset is dirty, expect more than the numbers of columns
df.get_dtype_counts() # object also means unidentified

object    6
dtype: int64

In [51]:
# In RegEx, this means <Selecting a digit four consecutive time, starting right away the begining of the text, while ignoring the bracket>
# DEFINITELY USE https://regex101.com/ as to understand RegEx (Regular Expression)
# https://stat545.com/block022_regular-expression.html

# Series.str.extract(pat, flags=0, expand=True)[source]
# Extract capture groups in the regex pat as columns in a DataFrame.
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
df.loc[:,'Date of Publication'] = pd.to_numeric(extr)

#Alternatively:  use re.findall, then iterate through the result
# set = df['Date of Publication'].astype(str).apply(lambda x: re.findall(r'^(\d{4})', x));
# sx = set.apply(lambda x: x[0] if len(x)>0 else "")

df['Date of Publication'].head(5) #Show how Date of Publication has changed.

Identifier
206    1879.0
216    1868.0
218    1869.0
472    1851.0
480    1857.0
Name: Date of Publication, dtype: float64

In [52]:


# Count how many NUlls we have and how they affect our analysis
df['Date of Publication'].isnull().sum() / len(df)

0.11717147339205986

In [53]:
#Let's try to standardize the Place of Publication. Let's try to see what kind of values are there.
# detect whether "London is present in that cell" -> Turn into a boolean vector/mask

#Filter the larger one first, then use the construction [["name here"]] to get to the column.

london_filter = df['Place of Publication'].str.contains('London')
oxford_filter = df['Place of Publication'].str.contains('Oxford')

df['Place of Publication'] = np.where(london_filter, 'London', np.where(oxford_filter, 'Oxford', df['Place of Publication'].str.replace('-', ' ')))
df[['Place of Publication','Publisher']].head(10)

,Place of Publication,Publisher
Identifier,,
206,London,S. Tinsley & Co.
216,London,Virtue & Co.
218,London,"Bradbury, Evans & Co."
472,London,James Darling
480,London,Wertheim & Macintosh
481,London,William Macintosh
519,London,The Author
667,Oxford,NaN
874,London,NaN


In [54]:
df2 = pd.read_csv('olympics.csv')
# At first, the starts one step above everything -> Column names do not match.
df2.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


In [55]:
# Set header =1 (Python starts at 0) to use this row as the column names
df2 = df2 = pd.read_csv('olympics.csv', header=1)
df2.head(5)

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [56]:
#Prepare a list of new column names, so that we can do this wholesale
new_names =  {'Unnamed: 0': 'Country',
               '? Summer': 'Summer Olympics',
               '01 !': 'Gold',
               '02 !': 'Silver',
               '03 !': 'Bronze',
               '? Winter': 'Winter Olympics',
               '01 !.1': 'Gold.1',
               '02 !.1': 'Silver.1',
               '03 !.1': 'Bronze.1',
               '? Games': '# Games',
               '01 !.2': 'Gold.2',
               '02 !.2': 'Silver.2',
               '03 !.2': 'Bronze.2'}

df2.rename(columns=new_names, inplace=True)
df2.head(5)

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
